In [1]:
from src.wrds_pull import fetch_taq_data
from src.strategy import OBIVWAPStrategy
from src.plot import plot_account_balance
from src.performance import evaluate_strategy_performance
import random
import polars as pl
import gc

EX_FILTER = "'Z'"
# EX_FILTER = exchanges = "'Q', 'N', 'T', 'P', 'Z', 'B', 'C', 'X', 'Y', 'A', 'M', 'D', 'I', 'J', 'K', 'L', 'V', 'W', 'U', 'S', 'F', 'G', 'H', 'O', 'R'"
QU_COND_FILTER = "'R'"
START_DATE = '2023-05-10'
END_DATE = '2023-05-11'
START_TIME = (9, 55)
END_TIME = (15, 36)
VWAP_WINDOW = 500
OBI_THRESHOLD = 0
SIZE_THRESHOLD = 0
VWAP_THRESHOLD = 0

In [2]:
# Choose a batch of tickers (example: first 2 from your filtered list)
with open("data/positive_return_tickers_v1.txt") as f:
    all_filtered = [line.strip() for line in f if line.strip()]

batch = random.sample(all_filtered, 2)
# batch = all_filtered[0:5] 
print("Batch tickers:", batch)

Batch tickers: ['WRB', 'ZION']


In [3]:
# Fetch TAQ data for this batch
df = fetch_taq_data(
    tickers=batch,
    exchanges=EX_FILTER,
    quote_conds=QU_COND_FILTER,
    start_date=START_DATE,
    end_date=END_DATE,
    wrds_username='changjulian17'
)
df

Loading library list...
Done


date,time_m,time_m_nano,ex,sym_root,sym_suffix,bid,bidsiz,ask,asksiz,qu_cond,qu_seqnum,natbbo_ind,qu_cancel,qu_source,rpi,ssr,luld_bbo_indicator,finra_bbo_ind,finra_adf_mpid,finra_adf_time,finra_adf_time_nano,finra_adf_mpq_ind,finra_adf_mquo_ind,sip_message_id,natl_bbo_luld,part_time,part_time_nano,secstat_ind,Timestamp
str,str,i64,str,str,str,f64,i64,f64,i64,str,i64,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,i64,str,datetime[ns]
"""2023-05-10""","""09:30:00.005927""",36,"""Z""","""ZION""",null,24.85,5,25.39,6,"""R""",4082573,"""2""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,null,"""09:30:00.005739""",0,null,2023-05-10 09:30:00.005927360
"""2023-05-10""","""09:30:00.006025""",617,"""Z""","""ZION""",null,24.85,5,25.39,7,"""R""",4082580,"""2""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,null,"""09:30:00.005839""",0,null,2023-05-10 09:30:00.006025617
"""2023-05-10""","""09:30:00.014506""",224,"""Z""","""ZION""",null,24.85,5,25.39,8,"""R""",4083130,"""0""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,null,"""09:30:00.014324""",0,null,2023-05-10 09:30:00.014506224
"""2023-05-10""","""09:30:00.025596""",499,"""Z""","""ZION""",null,24.85,5,25.39,7,"""R""",4084178,"""0""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,null,"""09:30:00.025411""",0,null,2023-05-10 09:30:00.025596499
"""2023-05-10""","""09:30:00.030080""",469,"""Z""","""ZION""",null,24.85,5,25.39,2,"""R""",4084586,"""0""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,null,"""09:30:00.029894""",0,null,2023-05-10 09:30:00.030080469
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2023-05-11""","""15:59:59.903834""",674,"""Z""","""ZION""",null,22.64,1,22.75,8,"""R""",88795359,"""0""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,"""A""","""15:59:59.903649""",0,null,2023-05-11 15:59:59.903834674
"""2023-05-11""","""15:59:59.906925""",919,"""Z""","""ZION""",null,22.65,2,22.75,8,"""R""",88795465,"""0""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,"""A""","""15:59:59.906733""",0,null,2023-05-11 15:59:59.906925919
"""2023-05-11""","""15:59:59.906957""",443,"""Z""","""ZION""",null,22.65,2,22.74,2,"""R""",88795466,"""0""",null,"""N""",null,"""0""",null,null,null,null,0,null,null,null,"""A""","""15:59:59.906772""",0,null,2023-05-11 15:59:59.906957443


In [4]:
import pandas as pd

batch_results = []

stock_tickers = df["sym_root"].unique().to_list()
print("Tickers in data:", stock_tickers)

for ticker in stock_tickers:
    print(f"Processing {ticker}...")
    ticker_data = df.filter(pl.col("sym_root") == ticker)
    strategy = OBIVWAPStrategy(
        vwap_window=VWAP_WINDOW, 
        obi_threshold=OBI_THRESHOLD, 
        size_threshold=SIZE_THRESHOLD,
        vwap_threshold=VWAP_THRESHOLD,
        start_time=START_TIME, 
        end_time=END_TIME
    )
    ticker_data = strategy.generate_signals(ticker_data)
    backtest_data = strategy.backtest(ticker_data)
    metrics = evaluate_strategy_performance(backtest_data)
    batch_results.append({
        "ticker": ticker,
        "Total_Returns": metrics.get("Total_Returns"),
        "Average_Sharpe": metrics.get("Average_Sharpe"),
        "Average_Bid_Ask_Spread": metrics.get("Average_Bid_Ask_Spread"),
        "Cumulative_Trades": metrics.get("Cumulative_Trades"),
        # Add more metrics here if needed
    })
    del ticker_data, backtest_data
    gc.collect()

results_df = pd.DataFrame(batch_results)
print(results_df)
# Optionally save to CSV
results_df.to_csv("data/one_batch_metrics.csv", index=False)

Tickers in data: ['ZION', 'WRB']
Processing ZION...

PERFORMANCE ZION STATISTICS:
Total returns: -99.95%
Max drawdown: -99.95%
Average bid-ask spread: 0.137435

DAILY SHARPE RATIOS:
shape: (2, 2)
┌─────────────────────┬────────────────────┐
│ Timestamp           ┆ Daily_Sharpe_Ratio │
│ ---                 ┆ ---                │
│ datetime[ns]        ┆ f64                │
╞═════════════════════╪════════════════════╡
│ 2023-05-10 00:00:00 ┆ -1.658043          │
│ 2023-05-11 00:00:00 ┆ -0.135787          │
└─────────────────────┴────────────────────┘
Processing WRB...

PERFORMANCE WRB STATISTICS:
Total returns: 2613.77%
Max drawdown: -9.26%
Average bid-ask spread: 0.225560

DAILY SHARPE RATIOS:
shape: (2, 2)
┌─────────────────────┬────────────────────┐
│ Timestamp           ┆ Daily_Sharpe_Ratio │
│ ---                 ┆ ---                │
│ datetime[ns]        ┆ f64                │
╞═════════════════════╪════════════════════╡
│ 2023-05-10 00:00:00 ┆ 0.77313            │
│ 2023-05-11 